# K-Adapter
## A standard workflow

In [ ]:
from kadapter.configurations import AdapterConfig, KAdapterSumHeadConfig, KAdapterConcatHeadConfig, KAdapterConfig
from kadapter.model import KAdapterModel

from transformers import RobertaModel, AutoModel, PretrainedConfig

%load_ext autoreload
%autoreload 2

Load an architecture of a single adapter and an arbitrary head from configurations (In case of one adapter, the heads are identical).
Also load a pretrained basemodel (in our case roberta-base) to inject the K-Adapter into.
As known from the Huggingface-Framework, the configurations can be changed in-place.

In [ ]:
fac_adapter_config = AdapterConfig('fac-adapter', hidden_dimension=768)
head_config = KAdapterSumHeadConfig()

basemodel = RobertaModel.from_pretrained('roberta-base', output_hidden_states=True)

kadapter_config = KAdapterConfig.from_adapter_configs(
    basemodel=basemodel.config, 
    adapters=[fac_adapter_config], 
    head=head_config,
    freeze_basemodel=True  # in order to pretrain the adapter
    )
model = KAdapterModel(config=kadapter_config, basemodel=basemodel)

Now that the KAdapter-Model has been loaded with a single Adapter, we can pretrain this Adapter to a specific task (e.g. RelationClassification).
...
After training, we can save the pretrained adapter

In [ ]:
pretrained_adapter = model.adapters[0]
pretrained_adapter.save_pretrained('./save/pretrained_adapter')

Do this with each individual adapter.
In the end, combine them all in the KAdapter architecture, injecting the Adapter knowledge into the basemodel.
At this point, the basemodel and the head can be trained on the actual task.

In [ ]:
loaded_adapter = AutoModel.from_pretrained('./save/pretrained_adapter')
# ... Load an arbitrary amount of pretrained adapters

# Create head from config
head_config = KAdapterConcatHeadConfig(n_adapters=1, hidden_size=loaded_adapter.config.hidden_size)
# Create a partial kadapter config
partial_kadapter_config = KAdapterConfig.from_adapter_configs(
    head=head_config,
    freeze_basemodel=False
)

# Create the K-Adapter Model by partly loading pretrained models and loading model architectures from configs.
model = KAdapterModel.from_adapters_pretrained('roberta-base', ['./save/pretrained_adapter'], None,
                                               config=partial_kadapter_config,
                                               output_hidden_states=True)

Train the knowledge-injected model on a certain dataset and save it!